# December 13th

Finding the point of reflection on either (both?) the horizontal or vertical axis.
Julia supports 2 dimensional arrays which we want to use to create this

In [79]:
example = [
    "#.##..##.",
    "..#.##.#.",
    "##......#",
    "##......#",
    "..#.##.#.",
    "..##..##.",
    "#.#.##.#.",
    "",
    "#...##..#",
    "#....#..#",
    "..##..###",
    "#####.##.",
    "#####.##.",
    "..##..###",
    "#....#..#"
]

15-element Vector{String}:
 "#.##..##."
 "..#.##.#."
 "##......#"
 "##......#"
 "..#.##.#."
 "..##..##."
 "#.#.##.#."
 ""
 "#...##..#"
 "#....#..#"
 "..##..###"
 "#####.##."
 "#####.##."
 "..##..###"
 "#....#..#"

In [80]:
## turn this into an array of 2 dimensional arrays, use booleans for values
function parsecontents(contents)
    results = []
    block = []
    for content in contents
        if content == ""
            push!(results, hcat(block...))
            block = []
        else
            push!(block, [v == '.' for v in content])
        end
    end
    push!(results, hcat(block...))
    results
end

println(parsecontents(example))

Any[Bool[0 1 0 0 1 1 0; 1 1 0 0 1 1 1; 0 0 1 1 0 0 0; 0 1 1 1 1 0 1; 1 0 1 1 0 1 0; 1 0 1 1 0 1 0; 0 1 1 1 1 0 1; 0 0 1 1 0 0 0; 1 1 0 0 1 1 1], Bool[0 0 1 0 0 1 0; 1 1 1 0 0 1 1; 1 1 0 0 0 0 1; 1 1 0 0 0 0 1; 0 1 1 0 0 1 1; 0 0 1 1 1 1 0; 1 1 0 0 0 0 1; 1 1 0 0 0 0 1; 0 0 0 1 1 0 0]]


We can solve the reflection point iteratively with O(n) and use the dimension factor in Julia to evaluate everything

In [81]:
function findreflection(grid, d)
    ## don't go all the way to the end
    for i in firstindex(grid, d):(lastindex(grid, d)-1)
        ## candidate for reflection
        if selectdim(grid, d, i) == selectdim(grid, d, i + 1)
            ## go as far back and forward as possible to find reflected values
            if all(
                t -> t[1] == t[2],
                zip(
                    ## look back
                    map(
                        b -> selectdim(grid, d, b),
                        range(i - 1, firstindex(grid, d), step=-1)
                    ),
                    ## look forward
                    map(
                        f -> selectdim(grid, d, f),
                        range(i + 2, lastindex(grid, d))
                    )
                ))
                return i
            end
        end
    end
    return 0
end

println(findreflection(parsecontents(example)[1], 1))
println(findreflection(parsecontents(example)[1], 2))
println(findreflection(parsecontents(example)[2], 1))
println(findreflection(parsecontents(example)[2], 2))

5
0
0
4


Solve Part one

In [82]:
function partone(contents)
    parsed = parsecontents(contents)
    sum(
        grid -> findreflection(grid, 1) + findreflection(grid, 2) * 100,
        parsed
    )
end

partone(example) == 405

true

## Part Two

Oh no there's smudges!
There's exactly one smudge in each input we'll need to write a smudge comparator

In [83]:

function issmudged(patterna, patternb, allowedsmudges)
    let smudges = 0
        for (a, b) in zip(patterna, patternb)
            if a != b
                if smudges < allowedsmudges
                    smudges += 1
                else
                    return 0
                end
            end
        end
        if smudges == 0
            return 1
        elseif smudges == allowedsmudges
            return 2
        else
            return 0
        end
    end
end

issmudged([true, false, true, false], [false, false, true, false], 1)

function issmudgedreflection(grid, fold, d, allowedsmudges)
    let smudges = 0
        for r in
            map(t -> issmudged(t[1], t[2], allowedsmudges), zip(
            ## look back
            map(
                b -> selectdim(grid, d, b),
                range(fold, firstindex(grid, d), step=-1)
            ),
            ## look forward
            map(
                f -> selectdim(grid, d, f),
                range(fold + 1, lastindex(grid, d))
            )
        ))
            if r == 0
                return false
            elseif r == 2
                if smudges < allowedsmudges
                    smudges += 1
                else
                    return false
                end
            end
        end
        return smudges == allowedsmudges
    end
end

issmudgedreflection (generic function with 1 method)

In [84]:
function findreflection(grid, d, allowedsmudges)
    ## don't go all the way to the end
    for i in firstindex(grid, d):(lastindex(grid, d)-1)
        ## candidate for reflection
        if issmudged(selectdim(grid, d, i), selectdim(grid, d, i + 1), allowedsmudges) != 0
            ## go as far back and forward as possible to find reflected values
            if issmudgedreflection(grid, i, d, allowedsmudges)
                return i
            end
        end
    end
    return 0
end

println(findreflection(parsecontents(example)[1], 1))
println(findreflection(parsecontents(example)[1], 2))
println(findreflection(parsecontents(example)[2], 1))
println(findreflection(parsecontents(example)[2], 2))

5
0
0
4


In [85]:
function parttwo(contents)
    parsed = parsecontents(contents)
    sum(
        grid -> findreflection(grid, 1, 1) + findreflection(grid, 2, 1) * 100,
        parsed
    )
end

parttwo(example) == 400

true

## Results

In [86]:
include("./aoc.jl")

execute(13, partone, parttwo; keepempty=true)

27502
